## Packages

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException
import time
from bs4 import BeautifulSoup
import csv
from selenium.webdriver.chrome.options import Options
import requests
import time
import tqdm
import pandas as pd
import os
import json
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import pprint

# Outomated browsing

### Set up, login and search for C20

In [18]:
# SETUP

options = Options()
options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems
options.add_argument('--no-sandbox')  # Bypass OS security model
options.add_argument('--disable-gpu')  # Applicable to windows os only
options.add_argument('start-maximized')  # Open Browser in maximized mode
options.add_argument('--headless')  # Run in headless mode (no GUI)

# Start webdriver (Chrome i dette eksempel)
driver = webdriver.Chrome()

# Dine login oplysninger
login = "jvgronlund@yahoo.com"
password = "rebklemmer72"

In [20]:
# Go to login site
driver.get('https://www.berlingske.dk/login')

# Decline cookies
try:
    accept_cookies_button = driver.find_element(By.CSS_SELECTOR, 'button[id="CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"]')
    accept_cookies_button.click()
    time.sleep(2)
except Exception as e:
    print("Ingen cookie-popup fundet, fortsætter uden.")

# Find login-felterne og indtast login-oplysninger
email_field = driver.find_element(By.ID, 'email')
password_field = driver.find_element(By.ID, 'current-password')

email_field.send_keys(login)
password_field.send_keys(password)

# Find and click login button
login_button = driver.find_element(By.CSS_SELECTOR, 'button[type="submit"]')
login_button.click()
time.sleep(5)

Ingen cookie-popup fundet, fortsætter uden.


In [5]:
# Go to search results for C20
driver.get('https://www.berlingske.dk/search?query=c20')

# Wait for page to load
time.sleep(5)

### Clicking "show more" untill all results are loaded

In [16]:
# Set up variables
links = []
num_clicks = 0
max_clicks = 1000  # Set your desired max number of clicks

# Progress bar setup with tqdm
with tqdm(total=max_clicks, desc="Progress", unit="click") as pbar:
    while num_clicks < max_clicks:
        try:
            # Find and click the "Show More" button
            vis_mere = driver.find_element(By.CSS_SELECTOR, '.search-results__load-more')
            vis_mere.click()

            # Wait for content to load after clicking the button
            time.sleep(3)  # Adjust as needed for the loading time

            # Increment the number of clicks
            num_clicks += 1

            # Update the progress bar
            pbar.update(1)

            # Get the updated page source after clicking
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')

            # Find all article elements on the page
            articles = soup.find_all('div', class_='teaser teaser--border-bottom teaser--search d-flex flex-row w-100')

            # Save links to all articles on the current page
            for article in articles:
                link = article.find('a', class_='teaser__title-link')['href']
                full_link = 'https://www.berlingske.dk' + link
                if full_link not in links:  # Avoid duplicates
                    links.append(full_link)

        except Exception as e:
            print("No more results to load or an error occurred:", e)
            break

# Close the browser after scraping
driver.quit()

print(f"Total links scraped: {len(links)}")


Progress:   0%|                                     | 0/1000 [00:00<?, ?click/s]

No more results to load or an error occurred: HTTPConnectionPool(host='localhost', port=60041): Max retries exceeded with url: /session/d7d992da77042959384300d5c6d4a81d/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x165f47680>: Failed to establish a new connection: [Errno 61] Connection refused'))
Total links scraped: 0


# Testing text extraction:

In [26]:
# Convert list to a pandas DataFrame
links_df = pd.DataFrame(links, columns=['urls'])

# Save DataFrame to CSV
links_df.to_csv('Berlingske_article_links.csv', index=False)

In [9]:
test_links = links[1300:1305]

In [10]:
test_links

['https://www.berlingske.dk/business/amerikanske-aktier-dykker-efter-trumps-soen-fremlaegger-e-mails',
 'https://www.berlingske.dk/business/renten-rykker-tilbage-til-udgangspunktet',
 'https://www.berlingske.dk/oekonomi/regnskab-foerte-tryg-frem-paa-boersen-maersk-igen-i-c20-top',
 'https://www.berlingske.dk/oekonomi/snapchats-aktier-dykker-efter-nedjustering-fra-analytiker',
 'https://www.berlingske.dk/oekonomi/pepsi-i-kursfald-trods-overraskelse-indeks-peger-ned']

In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# List of article URLs
urls = test_links

# Initialize an empty dictionary to store the scraped data
scraped_articles = {}

# Loop through each URL and extract the article text
for url in urls:
    try:
        # Open the webpage
        driver.get(url)

        # Wait for the page to load (adjust if needed)
        time.sleep(5)

        # Extract the main article text by targeting the correct HTML tag and class
        article_body = driver.find_element(By.ID, "articleBody")
        paragraphs = article_body.find_elements(By.TAG_NAME, "p")

        # Combine the text from all paragraphs
        article_text = "\n".join([paragraph.text for paragraph in paragraphs])

        # Store the article text using the URL as the key
        scraped_articles[url] = article_text
        print(f"Scraped article from {url}")
    except Exception as e:
        print(f"Failed to scrape {url}: {e}")

# Close the WebDriver
driver.quit()


Scraped article from https://www.berlingske.dk/business/amerikanske-aktier-dykker-efter-trumps-soen-fremlaegger-e-mails
Scraped article from https://www.berlingske.dk/business/renten-rykker-tilbage-til-udgangspunktet
Scraped article from https://www.berlingske.dk/oekonomi/regnskab-foerte-tryg-frem-paa-boersen-maersk-igen-i-c20-top
Scraped article from https://www.berlingske.dk/oekonomi/snapchats-aktier-dykker-efter-nedjustering-fra-analytiker
Scraped article from https://www.berlingske.dk/oekonomi/pepsi-i-kursfald-trods-overraskelse-indeks-peger-ned


In [25]:
scraped_articles

{'https://www.berlingske.dk/business/amerikanske-aktier-dykker-efter-trumps-soen-fremlaegger-e-mails': 'De amerikanske aktier falder, efter USA\'s præsident Donald Trumps søn Donald Trump Jr. har fremlagt en e-mail-korrespondance med briten Rob Goldstone, der satte et møde op mellem Donald Trumps søn og en russisk advokat.\nE-mail-korrespondancen viser umiddelbart, at Donald Trumps kampagnehold var i kontakt med Rusland, og at kampagnen var vidende om, at Rusland ville hjælpe Donald Trump til at vinde.\n»Dette her er åbenlyst på et meget højt niveau og ekstremt fortrolig information, men er den del af Rusland og dets regerings støtte til Mr. Trump,« skriver Rob Goldstone i en mail, hvor han er ved at sætte et møde op mellem Donald Trump Jr. og en russisk advokat.\n\nDonald Trump Jr. har ifølge sig selv lagt e-mailene frem for at give "total transparens" i forhold til omstændighederne for mødet. Selve mødet blev afsløret tidligere på ugen i amerikanske medier.\nEfter nyheden falder det 